In [ ]:
import codecs

session = spark.builder.getOrCreate();

storage_account_name = "#STORAGE_ACCOUNT_NAME#"
storage_account_access_key = "#STORAGE_ACCOUNT_KEY#"

connection_string = "DefaultEndpointsProtocol=https;AccountName=" + storage_account_name + ";AccountKey=" + storage_account_access_key + ";EndpointSuffix=core.windows.net"

session.conf.set(
	"fs.azure.account.key." + storage_account_name + ".blob.core.windows.net",
	storage_account_access_key
)

sdf = session.read.text(
	"wasbs://models@" + storage_account_name + ".blob.core.windows.net/onnx/model.onnx"
)

str3 = ''

def compare(a, b, encoding="utf8"):
    if isinstance(a, bytes):
        a = a.decode(encoding)
    if isinstance(b, bytes):
        b = b.decode(encoding)
    return a == b

for xs in sdf.take(sdf.count()):
    for x in xs:
        for l in x:
            try:
                arr = bytes(l, 'utf-8')

                newArr = codecs.getencoder('hex')(arr)[0]

                if compare(newArr, b'efbfbd'):
                    newArr = b'b4'

                str3 = str3 + newArr.decode("utf-8")
                
            except:
                print('Skipping' + l)

print(str3)

from azure.storage.blob import BlobClient

blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="models", blob_name="onnx/model.onnx.hex")
blob.upload_blob(str3,overwrite=True)